In [46]:
from collections import defaultdict
import numpy as np

# Step 1: Define the tree structure for the activation functions
activation_tree = {
    "root": ["unbounded", "bounded"],
    "unbounded": ["straight", "round"],
    "straight": ["relu", "leakyrelu"],
    "round": ["elu","gelu"],
    "bounded": ["tanh", "sigmoid"]
}

In [47]:
# Step 2: Build a mapping from child to parent for easy traversal
parent_map = {}
def build_parent_map(node, parent, tree):
    parent_map[node] = parent
    if node in tree:
        for child in tree[node]:
            build_parent_map(child, node, tree)

build_parent_map("root", None, activation_tree)

In [48]:
# Step 3: Function to calculate the distance (hops) between two activation functions
def find_distance(node1, node2, parent_map):
    # Find the path to the root for both nodes
    path1, path2 = [], []
    
    while node1 is not None:
        path1.append(node1)
        node1 = parent_map[node1]
    
    while node2 is not None:
        path2.append(node2)
        node2 = parent_map[node2]
    
    # Find the least common ancestor (LCA) by comparing the paths
    path1.reverse()
    path2.reverse()
    min_len = min(len(path1), len(path2))
    
    lca_distance = 0
    for i in range(min_len):
        if path1[i] == path2[i]:
            lca_distance = i
        else:
            break
    
    # The number of hops is the total distance to the LCA
    distance = (len(path1) - lca_distance - 1) + (len(path2) - lca_distance - 1)
    # return np.log2(distance)
    return distance

# Step 4: Compute probabilities based on distances
def compute_probabilities_from_tree(current_function, parent_map, all_functions):
    # Compute the hops for each function relative to the current function
    distances = {func: find_distance(current_function, func, parent_map) for func in all_functions if func != current_function}
    
    # Compute raw probabilities using 1 / distance (or some distance function)
    raw_probs = {func: 1 / (2**dist) for func, dist in distances.items()}
    
    # Normalize the probabilities
    total = sum(raw_probs.values())
    probabilities = {func: prob / total for func, prob in raw_probs.items()}
    
    return probabilities

In [49]:
find_distance("relu", "leakyrelu", parent_map)

2

In [50]:
# Define all activation functions
all_functions = ["relu", "leakyrelu", "elu", "gelu", "tanh", "sigmoid"]

# Example: compute mutation probabilities from 'relu'
probabilities_from_relu = compute_probabilities_from_tree("relu", parent_map, all_functions)
for func, prob in probabilities_from_relu.items():
    print(f"{func}: {prob*100:.1f}")


leakyrelu: 57.1
elu: 14.3
gelu: 14.3
tanh: 7.1
sigmoid: 7.1
